# 计算图
梯度下降法需要计算损失函数对参数的偏导数，如果用链式法则对每个参数逐一求偏导，这是一个非常艰巨的任务！这是因为：
- 模型参数非常多
- 复合函数非常复杂
学习误差反向传播法有两种方式：一是基于计算式，这种方法严密且简洁，但是对数学功底要求比较高；二是基于计算图（Computational graph），比较直观，易于理解。目前各大深度学习框架如Tensorflow、PyTorch、Theano等都以计算图作为描述反向传播算法的基础。

为何用计算图解题？
1. 局部计算，无论全局是多么复杂的计算，都可以通过局部计算使各个节点致力于简单的计算，从而简化问题。
2. 利用计算图可以将中间的计算结果全部保存起来
3. 可以通过反向传播高效计算导数。

从左向右进行计算”是一种正方向上的传播，简称为正向传播（forward propagation）。正向传播是从计算图出发点到结束点的传播。而从图上看从右向左的传播，这种传播称为反向传播（backward propagation）。

## 链式法则
局部计算导数的原理，基于**链式法则（chain rule）**。以复合函数 $z={(x+y)}^2$为例，它由下面两个式子构成：
$$
\large 
\begin{equation}\begin{split} 
z&=t^2\\
t&=x+y
\end{split} \end{equation}
$$

对应计算图如下，计算图的反向传播从右到传播信号。反向传播的顺序是，先将节点的输入信号乘以节点的局部导数（偏导数），然后再传递给下一个节点。
![计算图](img/computational_graph.png)

## 加法
考察$z=x+y$，偏导数为：
$$
\begin{aligned}
\frac {\partial z} {\partial x} &= 1 \\
\frac {\partial z} {\partial y} &= 1
\end{aligned}
$$

加法节点的反向传播将上游的值原封不动地输出到下游。

In [2]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        return x + y

    def backward(self, dout):
        return dout, dout

## 乘法
考察$z=xy$，偏导数为：
$$
\begin{aligned}
\frac {\partial z} {\partial x} &= y \\
\frac {\partial z} {\partial y} &= x
\end{aligned}
$$
乘法的反向传播会将上游的值乘以正向传播时的输入信号的“翻转值”后传递给下游。

In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y

    def backward(self, dout):
        dx = dout * self.y # 翻转x和y
        dy = dout * self.x
        return dx, dy

## ReLU
$$
y=\begin{cases}
x,\quad (x \gt 0) \\
0,\quad (x \le 0)
\end{cases}
$$
偏导数为：
$$
\frac {\partial y} {\partial x}=\begin{cases}
1,\quad (x \gt 0) \\
0,\quad (x \le 0)
\end{cases}
$$

![relu](img/relu.png)

ReLU层的作用就像电路中的开关一样。正向传播时，有电流通过的话，就将开关设为ON；没有电流通过的话，就将开关设为OFF。反向传播时，开关为ON的话，电流会直接通过；开关为OFF的话，则不会有电流通过。

In [5]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # 生成bool数组
        out = x.copy()
        out[self.mask] = 0 # 将小于0的元素置为0
        return out

    def backward(self, dout):
        dout[self.mask] = 0 # 将小于0的元素置为0
        dx = dout
        return dx

import numpy as np

x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x <= 0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


## sigmoid
$$
y = \frac{1}{1 + e^{-x}}
$$

偏导数为
$$
\cfrac{\partial L}{\partial y}y^2e^{-x}
$$

![sigmoid](img/sigmoid.png)

此外，$\cfrac{\partial L}{\partial y}y^2e^{-x}$ 还可以进一步整理：

$$
\large
\begin{equation}\begin{split} 
\frac{\partial L}{\partial y}y^2e^{-x}&=\frac{\partial L}{\partial y}\frac{1}{\left(1+e^{-x}\right)^2}e^{-x} \\ 
&=\frac{\partial L}{\partial y}\frac{1}{1+e^{-x}}\frac{e^{-x}}{1+e^{-x}}\\ 
&=\frac{\partial L}{\partial y}y(1-y)
\end{split}\end{equation}\tag{5.12}
$$

因此，Sigmoid函数的反向传播，只根据正向传播的输出就能计算出来。sigmoid之所以被选为激活函数的一个很重要的原因也是因为其导数计算比较容易。

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x)) # sigmoid函数
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out # sigmoid函数的导数
        return dx

# Affine
Affine层就是全连接层，以线性代数的视角它进行的是一个点积和偏移操作，因此也叫仿射层。

$$
Y = X \cdot W + B
$$

以矩阵为对象的反向传播，按矩阵的各个元素计算时，步骤和标量为对象的情形一样。由此可以得到下式：
$$
\begin{equation}\begin{split} 
\frac{\partial L}{\partial \textbf{X}}&=\frac{\partial L}{\partial \textbf{Y}}\cdot {\textbf{W}}^{\rm T}\\[2ex]
\frac{\partial L}{\partial \textbf{W}}&={\textbf{X}}^{\rm T} \cdot \frac{\partial L}{\partial \textbf{Y}}
\end{split} \end{equation}
$$

由此我们可以推出`dot`节点计算图

![affine](img/affine.png)

考虑N个数据一起进行正向传播的情况，也就是批处理版本的Affine层。注意输入的$X$的形状是$(N, 2)$。
![affine](img/affine2.png)

注意$\frac {\partial L} {\partial B}$需要按列求和一下。

In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W # 权重
        self.b = b # 偏置
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x # 保存输入
        out = np.dot(x, self.W) + self.b # 矩阵乘法
        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T) # 矩阵乘法
        self.dW = np.dot(self.x.T, dout) # 矩阵乘法
        self.db = np.sum(dout, axis=0) # 按列求和
        return dx

# Softmax-with-Loss

softmax 函数会将输入值正规化之后再输出，在这里我们考虑包含交叉熵损失函数，所以称为“Softmax-with-Loss层”。

> 神经网络中进行的处理有推理（inference）和学习两个阶段。神经网络的推理通常不使用Softmax 层。神经网络中未被正规化的输出结果有时被称为“得分”。也就是说，当神经网络的推理只需要给出一个答案的情况下，因为此时只对得分最大值感兴趣，所以不需要Softmax 层。不过，神经网络的学习阶段则需要Softmax层。

![softmax](img/softmax2.png)

如上图所示，softmax 函数记为 Softmax 层，交叉熵误差集为 Cross Entropy Error 层。这里假设进行 3 分类，从前面的层接收 3 个输入。

Softmax 层将输入 $(a_1,a_2,a_3)$ 正规化，输出 $(y_1,y_2,y_3)$。 Cross Entropy Error 层接收 Softmax 的输出 $(y_1,y_2,y_3)$ 和监督标签 $(t_1,t_2,t_3)$，从这些数据中输出损失 $L$。

![softmax](img/softmax.png)

这里要注意的是反向传播的结果。Softmax 层的反向传播得到了 $(y_1-t_1,y_2-t_2,y_3-t_3)$ 这样的整齐漂亮结果。由于 $(y_1,y_2,y_3)$ 是 Softmax 层的输出，$(t_1,t_2,t_3)$ 是监督数据，所以 $(y_1-t_1,y_2-t_2,y_3-t_3)$ 是 Softmax 层的输出和监督标签的差分。神经网络的反向传播会把这个差分表示的误差传递给前面的层，这是神经网络学习中的重要性质。

神经网络学习的目的就是通过调整权重参数，使神经网络的输出（Softmax的输出）接近监督标签。因此，必须将神经网络的输出与监督标签的误差高效地传递给前面的层。刚刚的 $(y_1-t_1,y_2-t_2,y_3-t_3)$ 正是 Softmax 层与监督标签的差，直接了当地表示了当前神经网络地输出与监督标签的误差。

考虑一个具体的例子，比如考察监督标签是$(0, 1, 0)$，softmax层的输出是$(0.3, 0.2, 0.5)$的情形。因为正解标签处的概率是$0.2(20%)$，这个时候神经网络未能进行正确的识别。吃石化，softmax层的反向传播传递的是$(0.3, -0.8, 0.5)$这样一个大的误差。因为这个大的误差会向前面的层传播，所以softmax层前面的层会从这个大的误差中学习到“大”的内容。反之则是一个小的误差，学习到的内容也很“小”。

> 使用交叉熵误差作为softmax 函数的损失函数后，反向传播得到$(y_1 − t_1, y_2 − t_2, y_3 − t_3)$这样“ 漂亮”的结果。实际上，这样“漂亮”的结果并不是偶然的，而是为了得到这样的结果，**特意设计了交叉熵误差函数**。回归问题中输出层使用“恒等函数”，损失函数使用“平方和误差”，也是出于同样的理由。也就是说，使用“平方和误差”作为“恒等函数”的损失函数，反向传播才能得到这样“漂亮”的结果。

In [6]:
from common.functions import softmax, cross_entropy_error

class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 损失
        self.y = None # softmax的输出
        self.t = None # 监督数据（one-hot vector）

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x) # softmax函数
        self.loss = cross_entropy_error(self.y, self.t) # 交叉熵误差
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0] # batch的大小
        dx = (self.y - self.t) / batch_size # 对应元素相除
        return dx

# 误差反向传播方法实现

神经网络中有合适的权重和偏置，调整权重和偏置以拟合训练数据的过程叫做学习。神经网络的学习分为以下四个步骤。
- **步骤一（mini-batch）**

    从训练数据中随机选择一部分数据，大小为 batch-size。

- **步骤二（计算梯度）**

    计算损失函数关于各个可训练参数（权重和偏置）的梯度。

- **步骤三（更新参数）**

    将可训练参数沿着梯度方向进行微小的更新。
    
- **步骤四（重复）**

    重复步骤一、步骤二、步骤三，直到一定次数，或者损失不再下降为止。

之前介绍过的误差反向传播法会在步骤2中出现，之前我们是通过数值微分的方法求得这个梯度。数值微分实现虽然简单，但是计算耗费时间非常多，但是误差反向传播法可以高效快速地计算梯度，因为刚刚我们都是通过计算图求的解析解。

In [8]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
            weight_init_std=0.01):
        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * \
                np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 生成层
        self.layers = OrderedDict() # 注意是有序字典(它会记住添加顺序)
        self.layers['Affine1'] = \
                Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
                Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x): # 预测
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t): # 损失函数
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t): # 计算精度
        y = self.predict(x)
        y = np.argmax(y, axis=1) # 获取最大值的索引
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0]) # 计算精度
        return accuracy

    def numerical_gradient(self, x, t): # 计算梯度
        loss_W = lambda W: self.loss(x, t) # 损失函数
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 计算梯度
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) # 计算梯度
        grads['W2'] = numerical_gradient(loss_W, self.params['W2']) # 计算梯度
        grads['b2'] = numerical_gradient(loss_W, self.params['b2']) # 计算梯度

        return grads
    
    def gradient(self, x, t): # 计算梯度
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values()) # 获取层
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 设定
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000 # 适当设定循环的次数
train_size = x_train.shape[0] # 训练数据的大小
batch_size = 100 # mini-batch的大小
learning_rate = 0.1 # 学习率
train_loss_list = [] # 训练损失
train_acc_list = [] # 训练精度
test_acc_list = [] # 测试精度

iter_per_epoch = max(train_size / batch_size, 1) # 1个epoch的重复次数

for i in range(iters_num): # 随机梯度下降
    # 获取mini-batch
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 计算梯度
    grad = network.gradient(x_batch, t_batch)

    # 更新参数
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 记录学习过程
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 计算每个epoch的识别精度
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train) # 训练精度
        test_acc = network.accuracy(x_test, t_test) # 测试精度
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + \
                str(test_acc))

train acc, test acc | 0.10965, 0.1048
train acc, test acc | 0.9050833333333334, 0.9054
train acc, test acc | 0.9260833333333334, 0.9263
train acc, test acc | 0.93875, 0.9394
train acc, test acc | 0.9452166666666667, 0.9432
train acc, test acc | 0.95185, 0.9486
train acc, test acc | 0.9555666666666667, 0.9526
train acc, test acc | 0.9600833333333333, 0.957
train acc, test acc | 0.9634, 0.9588
train acc, test acc | 0.96515, 0.9589
train acc, test acc | 0.9682166666666666, 0.9621
train acc, test acc | 0.9708666666666667, 0.9627
train acc, test acc | 0.97175, 0.9634
train acc, test acc | 0.97375, 0.9665
train acc, test acc | 0.9757166666666667, 0.9679
train acc, test acc | 0.9758, 0.9685
train acc, test acc | 0.9782166666666666, 0.9691


# 梯度确认
确认数值微分求出的结果和误差反向传播法求出的结果是否一致（相近）的操作称为梯度确认(gradient check)。

In [9]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch) # 数值梯度
grad_backprop = network.gradient(x_batch, t_batch) # 反向传播梯度

# 求各个权重的绝对误差的平均值
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))


W1:4.084475604007241e-10
b1:2.0655231361085305e-09
W2:5.4918082193730924e-09
b2:1.3986450873582078e-07
